In [1]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
dataframe = fdr.DataReader('005930', '2000')     # Index is already 'Date', 데이터 불러오기
dataframe.dropna(inplace=True) # 결측치제거
dataframe

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2000-01-04,6000,6110,5660,6110,1483967,0.148496
2000-01-05,5800,6060,5520,5580,1493604,-0.086743
2000-01-06,5750,5780,5580,5620,1087810,0.007168
2000-01-07,5560,5670,5360,5540,806195,-0.014235
2000-01-10,5600,5770,5580,5770,937615,0.041516
...,...,...,...,...,...,...
2021-11-25,75100,75100,73600,73700,12559258,-0.014706
2021-11-26,73500,74100,72000,72300,13002242,-0.018996
2021-11-29,71700,73000,71400,72300,16682559,0.000000


In [3]:
def updown(dataframe):
    dataframe['UD_Trend'] = dataframe['Change'].map(lambda x : trend_separater(x)) 
    dataframe['UD_Trend'] = dataframe['UD_Trend'].shift(-1)  # 다음날 트렌드를 예측해야하므로 다음날 트렌드를 앞으로 한 행 당긴다
    dataframe.dropna(inplace=True)  # 결측치 제거
    return dataframe

def trend_separater(x):
    if x > 0.0015: # 상승과 하락 트렌드의 절대적인 개수를 비슷하게 맞춰주기 위한 기준점을 선정한다.
        return 1
    elif x < -0.000001:
        return -1
 

In [4]:
dataframe = updown(dataframe)
total_count = dataframe.UD_Trend.count()
dataframe['UD_Trend'].value_counts()

 1.0    2551
-1.0    2551
Name: UD_Trend, dtype: int64

In [5]:
target_df = dataframe.copy()
target_df.reset_index(inplace=True)
target_df

,Date,Open,High,Low,Close,Volume,Change,UD_Trend
0,2000-01-04,6000,6110,5660,6110,1483967,0.148496,-1.0
1,2000-01-05,5800,6060,5520,5580,1493604,-0.086743,1.0
2,2000-01-06,5750,5780,5580,5620,1087810,0.007168,-1.0
3,2000-01-07,5560,5670,5360,5540,806195,-0.014235,1.0
4,2000-01-11,5820,6100,5770,5770,1194974,0.000000,-1.0
...,...,...,...,...,...,...,...,...
5097,2021-11-23,76000,76000,74500,75300,22029195,0.005340,-1.0
5098,2021-11-24,76000,76200,74100,74800,15652305,-0.006640,-1.0
5099,2021-11-25,75100,75100,73600,73700,12559258,-0.014706,-1.0
5100,2021-11-29,71700,73000,71400,72300,16682559,0.000000,-1.0


In [6]:
# 전체 세트
X_all = np.array(target_df.loc[:, target_df.drop(['UD_Trend', 'Date'], axis=1).columns])
y_all = np.array(target_df.loc[:, ['UD_Trend']])

# 훈련 세트
X_train = X_all[:4000]
y_train = y_all[:4000]

# 검증 세트
X_test = X_all[4000:]
y_test = y_all[4000:]

In [7]:
svm_cla = Pipeline([
                    ("scaler", StandardScaler()),
                    ("svc", SVC()),
            ])

In [8]:
svm_cla.fit(X_train, y_train) # 훈련 세트로 훈련진행

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])

In [9]:
print("train_set score: ", svm_cla.score(X_train, y_train)) # 훈련세트 점수
print("test_set score : ", svm_cla.score(X_test, y_test)) # 검증세트 점수

train_set score:  0.5405
test_set score :  0.5009074410163339


In [10]:
sample = target_df.copy()
sample['predicted'] = svm_cla.predict(X_all)
sample.head(50)

,Date,Open,High,Low,Close,Volume,Change,UD_Trend,predicted
0,2000-01-04,6000,6110,5660,6110,1483967,0.148496,-1.0,-1.0
1,2000-01-05,5800,6060,5520,5580,1493604,-0.086743,1.0,-1.0
2,2000-01-06,5750,5780,5580,5620,1087810,0.007168,-1.0,1.0
3,2000-01-07,5560,5670,5360,5540,806195,-0.014235,1.0,-1.0
4,2000-01-11,5820,6100,5770,5770,1194974,0.000000,-1.0,-1.0
5,2000-01-12,5610,5740,5600,5720,584492,-0.008666,-1.0,1.0
6,2000-01-13,5600,5740,5560,5710,823830,-0.001748,1.0,1.0
7,2000-01-14,5720,5880,5680,5830,987576,0.021016,1.0,1.0
8,2000-01-18,6160,6160,5980,6100,905231,0.000000,-1.0,1.0
9,2000-01-19,6000,6040,5960,5960,804159,-0.022951,1.0,-1.0


In [11]:
def create_trade_book(sample):
    book = sample[['Close', 'UD_Trend', 'predicted', 'Date']].copy()
    book['trade'] = ''
    return book

def returns(book):    
    rtn = 1.0
    book['return'] = 1
    buy = 0.0
    sell = 0.0
    for i in book.index[1:]:
        if book.loc[i, 'trade'] == 'buy' and book.shift(1).loc[i, 'trade'] == '':
            buy = book.loc[i, 'Close']
            print('매수일 : ', book.loc[i, 'Date'].strftime('%Y-%m-%d'), '매수가격 : ', buy)
        elif book.loc[i, 'trade'] == '' and book.shift(1).loc[i, 'trade'] == 'buy':
            sell = book.loc[i, 'Close']
            rtn = (sell - buy) / buy + 1
            book.loc[i, 'return'] = rtn
            print('매도일 : ', book.loc[i, 'Date'].strftime('%Y-%m-%d'), '매도가격 : ', sell, ' | return:', round(rtn, 4))

        if book.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0

        acc_rtn = 1.0
    for i in book.index:
        rtn = book.loc[i, 'return']
        acc_rtn = acc_rtn * rtn
        book.loc[i, 'acc return'] = acc_rtn

    print('Accumulated return :', round(acc_rtn, 4))
    return(round(acc_rtn, 4))

def tradings(sample, book):
    # 다음날 오를것으로 예측했으면 매수, 내릴것으로 예측했으면 매도
    for i in sample.index[1:]:
        if book.shift(1).loc[i, 'trade'] == 'buy':
            if book.loc[i, 'predicted'] == -1:
                book.loc[i, 'trade'] = ''
            else:
                book.loc[i, 'trade'] = 'buy'
                             
        else:
            if book.loc[i, 'predicted'] == 1:
                book.loc[i, 'trade'] = 'buy'
    return(book)

In [ ]:
book = create_trade_book(sample) # 거래장부
tradings(sample, book) # 거래 실행
book.tail(100)

In [13]:
returns(book) # 수익률 계산

매수일 :  2000-01-06 매수가격 :  5620
매도일 :  2000-01-07 매도가격 :  5540  | return: 0.9858
매수일 :  2000-01-12 매수가격 :  5720
매도일 :  2000-01-19 매도가격 :  5960  | return: 1.042
매수일 :  2000-01-20 매수가격 :  6040
매도일 :  2000-01-21 매도가격 :  5880  | return: 0.9735
매수일 :  2000-01-26 매수가격 :  5480
매도일 :  2000-01-31 매도가격 :  5580  | return: 1.0182
매수일 :  2000-02-03 매수가격 :  5130
매도일 :  2000-02-11 매도가격 :  5700  | return: 1.1111
매수일 :  2000-02-17 매수가격 :  5080
매도일 :  2000-02-18 매도가격 :  5300  | return: 1.0433
매수일 :  2000-02-22 매수가격 :  5120
매도일 :  2000-02-24 매도가격 :  5070  | return: 0.9902
매수일 :  2000-02-25 매수가격 :  5120
매도일 :  2000-02-28 매도가격 :  4800  | return: 0.9375
매수일 :  2000-02-29 매수가격 :  5120
매도일 :  2000-03-06 매도가격 :  5640  | return: 1.1016
매수일 :  2000-03-07 매수가격 :  5620
매도일 :  2000-03-09 매도가격 :  5620  | return: 1.0
매수일 :  2000-03-14 매수가격 :  5210
매도일 :  2000-03-23 매도가격 :  6200  | return: 1.19
매수일 :  2000-03-28 매수가격 :  7030
매도일 :  2000-03-29 매도가격 :  7660  | return: 1.0896
매수일 :  2000-04-03 매수가격 :  7100
매도일 :  2000-04-

매도일 :  2002-01-28 매도가격 :  6370  | return: 1.0617
매수일 :  2002-02-01 매수가격 :  6180
매도일 :  2002-02-06 매도가격 :  6380  | return: 1.0324
매수일 :  2002-02-08 매수가격 :  6420
매도일 :  2002-02-14 매도가격 :  7100  | return: 1.1059
매수일 :  2002-02-18 매수가격 :  6860
매도일 :  2002-02-21 매도가격 :  6760  | return: 0.9854
매수일 :  2002-02-25 매수가격 :  6640
매도일 :  2002-02-28 매도가격 :  6860  | return: 1.0331
매수일 :  2002-03-04 매수가격 :  7000
매도일 :  2002-03-07 매도가격 :  7120  | return: 1.0171
매수일 :  2002-03-11 매수가격 :  6970
매도일 :  2002-03-13 매도가격 :  6960  | return: 0.9986
매수일 :  2002-03-18 매수가격 :  6640
매도일 :  2002-03-21 매도가격 :  6830  | return: 1.0286
매수일 :  2002-03-22 매수가격 :  7010
매도일 :  2002-03-25 매도가격 :  6800  | return: 0.97
매수일 :  2002-03-26 매수가격 :  6980
매도일 :  2002-03-28 매도가격 :  7150  | return: 1.0244
매수일 :  2002-03-29 매수가격 :  7360
매도일 :  2002-04-02 매도가격 :  7970  | return: 1.0829
매수일 :  2002-04-03 매수가격 :  8080
매도일 :  2002-04-04 매도가격 :  7780  | return: 0.9629
매수일 :  2002-04-12 매수가격 :  7380
매도일 :  2002-04-18 매도가격 :  8130  | return: 

매도일 :  2004-03-18 매도가격 :  10880  | return: 1.0362
매수일 :  2004-03-19 매수가격 :  11100
매도일 :  2004-03-22 매도가격 :  10880  | return: 0.9802
매수일 :  2004-03-24 매수가격 :  10840
매도일 :  2004-03-25 매도가격 :  10700  | return: 0.9871
매수일 :  2004-03-26 매수가격 :  10900
매도일 :  2004-04-02 매도가격 :  11300  | return: 1.0367
매수일 :  2004-04-06 매수가격 :  11900
매도일 :  2004-04-09 매도가격 :  11900  | return: 1.0
매수일 :  2004-04-12 매수가격 :  12200
매도일 :  2004-04-14 매도가격 :  12300  | return: 1.0082
매수일 :  2004-04-19 매수가격 :  11980
매도일 :  2004-04-22 매도가격 :  12400  | return: 1.0351
매수일 :  2004-04-23 매수가격 :  12740
매도일 :  2004-04-26 매도가격 :  12520  | return: 0.9827
매수일 :  2004-05-07 매수가격 :  10800
매도일 :  2004-05-10 매도가격 :  10180  | return: 0.9426
매수일 :  2004-05-12 매수가격 :  10500
매도일 :  2004-05-13 매도가격 :  10140  | return: 0.9657
매수일 :  2004-05-18 매수가격 :  9470
매도일 :  2004-05-20 매도가격 :  9890  | return: 1.0444
매수일 :  2004-05-21 매수가격 :  10040
매도일 :  2004-05-25 매도가격 :  9840  | return: 0.9801
매수일 :  2004-05-27 매수가격 :  10120
매도일 :  2004-05-31 매도가격

매수일 :  2006-05-17 매수가격 :  13060
매도일 :  2006-05-18 매도가격 :  12820  | return: 0.9816
매수일 :  2006-05-23 매수가격 :  12600
매도일 :  2006-05-24 매도가격 :  12580  | return: 0.9984
매수일 :  2006-05-26 매수가격 :  12400
매도일 :  2006-05-29 매도가격 :  12440  | return: 1.0032
매수일 :  2006-06-02 매수가격 :  12280
매도일 :  2006-06-05 매도가격 :  12260  | return: 0.9984
매수일 :  2006-06-16 매수가격 :  11420
매도일 :  2006-06-19 매도가격 :  11280  | return: 0.9877
매수일 :  2006-06-22 매수가격 :  11300
매도일 :  2006-06-27 매도가격 :  11660  | return: 1.0319
매수일 :  2006-06-29 매수가격 :  11800
매도일 :  2006-07-03 매도가격 :  12100  | return: 1.0254
매수일 :  2006-07-10 매수가격 :  12120
매도일 :  2006-07-12 매도가격 :  12160  | return: 1.0033
매수일 :  2006-07-20 매수가격 :  11960
매도일 :  2006-07-21 매도가격 :  11940  | return: 0.9983
매수일 :  2006-07-25 매수가격 :  11960
매도일 :  2006-07-26 매도가격 :  11980  | return: 1.0017
매수일 :  2006-07-27 매수가격 :  12080
매도일 :  2006-07-28 매도가격 :  12000  | return: 0.9934
매수일 :  2006-07-31 매수가격 :  12160
매도일 :  2006-08-01 매도가격 :  12100  | return: 0.9951
매수일 :  2006-08-0

매도일 :  2008-09-09 매도가격 :  10720  | return: 1.0308
매수일 :  2008-09-10 매수가격 :  10840
매도일 :  2008-09-11 매도가격 :  10740  | return: 0.9908
매수일 :  2008-09-12 매수가격 :  10860
매도일 :  2008-09-18 매도가격 :  10340  | return: 0.9521
매수일 :  2008-09-22 매수가격 :  11160
매도일 :  2008-09-26 매도가격 :  11060  | return: 0.991
매수일 :  2008-10-07 매수가격 :  10620
매도일 :  2008-10-08 매도가격 :  10560  | return: 0.9944
매수일 :  2008-10-09 매수가격 :  10820
매도일 :  2008-10-10 매도가격 :  10420  | return: 0.963
매수일 :  2008-10-13 매수가격 :  10480
매도일 :  2008-10-15 매도가격 :  10940  | return: 1.0439
매수일 :  2008-10-16 매수가격 :  10080
매도일 :  2008-10-17 매도가격 :  10100  | return: 1.002
매수일 :  2008-10-20 매수가격 :  10420
매도일 :  2008-10-21 매도가격 :  10380  | return: 0.9962
매수일 :  2008-10-24 매수가격 :  8150
매도일 :  2008-10-29 매도가격 :  9500  | return: 1.1656
매수일 :  2008-11-12 매수가격 :  9610
매도일 :  2008-11-13 매도가격 :  9480  | return: 0.9865
매수일 :  2008-11-19 매수가격 :  8760
매도일 :  2008-11-20 매도가격 :  8460  | return: 0.9658
매수일 :  2008-11-21 매수가격 :  8840
매도일 :  2008-12-01 매도가격 :  

매수일 :  2011-11-04 매수가격 :  20100
매도일 :  2011-11-07 매도가격 :  19860  | return: 0.9881
매수일 :  2011-11-10 매수가격 :  18700
매도일 :  2011-11-15 매도가격 :  19920  | return: 1.0652
매수일 :  2011-12-01 매수가격 :  21480
매도일 :  2011-12-02 매도가격 :  21000  | return: 0.9777
매수일 :  2011-12-06 매수가격 :  20880
매도일 :  2011-12-07 매도가격 :  21120  | return: 1.0115
매수일 :  2011-12-13 매수가격 :  21000
매도일 :  2011-12-14 매도가격 :  20800  | return: 0.9905
매수일 :  2011-12-19 매수가격 :  20140
매도일 :  2011-12-20 매도가격 :  20240  | return: 1.005
매수일 :  2011-12-21 매수가격 :  21140
매도일 :  2011-12-22 매도가격 :  21040  | return: 0.9953
매수일 :  2012-01-04 매수가격 :  21600
매도일 :  2012-01-06 매도가격 :  20800  | return: 0.963
매수일 :  2012-01-19 매수가격 :  21440
매도일 :  2012-01-20 매도가격 :  22100  | return: 1.0308
매수일 :  2012-02-23 매수가격 :  23200
매도일 :  2012-02-24 매도가격 :  23600  | return: 1.0172
매수일 :  2012-02-27 매수가격 :  23420
매도일 :  2012-02-28 매도가격 :  23700  | return: 1.012
매수일 :  2012-03-02 매수가격 :  23640
매도일 :  2012-03-06 매도가격 :  23600  | return: 0.9983
매수일 :  2012-03-07 매

매도일 :  2014-10-06 매도가격 :  23020  | return: 0.9721
매수일 :  2014-10-08 매수가격 :  22600
매도일 :  2014-10-10 매도가격 :  22100  | return: 0.9779
매수일 :  2014-10-17 매수가격 :  21780
매도일 :  2014-10-20 매도가격 :  22120  | return: 1.0156
매수일 :  2014-10-21 매수가격 :  21660
매도일 :  2014-10-22 매도가격 :  22080  | return: 1.0194
매수일 :  2014-10-23 매수가격 :  21880
매도일 :  2014-10-24 매도가격 :  22240  | return: 1.0165
매수일 :  2014-10-27 매수가격 :  21900
매도일 :  2014-10-29 매도가격 :  22600  | return: 1.032
매수일 :  2014-10-30 매수가격 :  23620
매도일 :  2014-11-17 매도가격 :  24100  | return: 1.0203
매수일 :  2014-11-19 매수가격 :  24360
매도일 :  2014-11-21 매도가격 :  24460  | return: 1.0041
매수일 :  2014-11-24 매수가격 :  24400
매도일 :  2014-11-26 매도가격 :  24020  | return: 0.9844
매수일 :  2014-11-27 매수가격 :  25280
매도일 :  2014-12-09 매도가격 :  26980  | return: 1.0672
매수일 :  2014-12-10 매수가격 :  26340
매도일 :  2014-12-11 매도가격 :  26100  | return: 0.9909
매수일 :  2014-12-12 매수가격 :  25720
매도일 :  2014-12-16 매도가격 :  25580  | return: 0.9946
매수일 :  2014-12-18 매수가격 :  25320
매도일 :  2014-12-22

163.7941